# Creating of bounding box files for use with Classifier Training

### Pre-generating the cropped inputs with our mask model:
To do this we will save the bounding boxes to a .csv and use it when training the resnet to crop the images

In [1]:
#imports:
from torchvision.models.detection import maskrcnn_resnet50_fpn_v2
import torch
import pandas as pd
import os
from dataloader import ISICMaskImageDataset
from torch.utils.data import DataLoader
batch_size = 1
dataset = "Dataset_2017\\"
data_aug_type = "2" #what data augmentation schema to train on
mask_model_path = 'Trained_Models\\Mask-RCNN_Best_{}'.format(data_aug_type)

device = "cuda"
device = torch.device(device)

In [2]:
import csv
def writeCSV(model,dataloader,device,dataset,datasetType,data_aug_type):
    filepath = "{}\\Pred_bb\\{}_bb_{}.csv".format(dataset, datasetType, data_aug_type)
    if(datasetType == "Training"):
        data_aug_type_label = data_aug_type
    else:
        data_aug_type_label = "1"
    img_labels = pd.read_csv(os.path.join(dataset,os.path.join("Ground_Truths", datasetType+"_GroundTruth_"+data_aug_type_label+".csv")))
    with open(filepath, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["image_id","x1","y1","x2","y2"])
        model.eval()

        # Disable gradient computation and reduce memory consumption.
        with torch.no_grad():
            for i,data  in enumerate(dataloader):

                vinputs, vtargets = data#next(iter(dataloader))#vdata
                #vtargets = move_data_to_device(vtargets, device)
                with torch.amp.autocast(torch.device(device).type):
                    outputs = model(torch.stack(vinputs).to(device))
                    for j, out in enumerate(outputs):
                        #print(out)
                        #print("{}".format(img_labels.iloc[i*batch_size+j, 0]))
                        #print(i*batch_size+j)
                        bb = out["boxes"][out["scores"].argmax(0).item()].tolist()
                        writer.writerow(["{}".format(img_labels.iloc[i*batch_size+j, 0]),bb[0],bb[1],bb[2],bb[3]])

In [3]:
mask_model = maskrcnn_resnet50_fpn_v2()
checkpoint = torch.load(mask_model_path)
mask_model.load_state_dict(checkpoint)
mask_model.to(device)

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         

In [4]:
datasetType = "Test"

Train_Dataset      = ISICMaskImageDataset(dataset, "Training",data_aug_type=data_aug_type)
Validation_Dataset = ISICMaskImageDataset(dataset, "Validation", data_aug_type="1")
Test_Dataset       = ISICMaskImageDataset(dataset, "Test", data_aug_type="1")

data_loader_params = {
    'batch_size': batch_size,  # Batch size for data loading
    'num_workers': 0,  # Number of subprocesses to use for data loading
    'persistent_workers': False,  # If True, the data loader will not shutdown the worker processes after a dataset has been consumed once. This allows to maintain the worker dataset instances alive.
    'pin_memory': True,  # If True, the data loader will copy Tensors into CUDA pinned memory before returning them. Useful when using GPU.
    'pin_memory_device': 'cuda' ,  # Specifies the device where the data should be loaded. Commonly set to use the GPU.
    'collate_fn': lambda batch: tuple(zip(*batch)),
}
train_dataloader      = DataLoader(Train_Dataset, **data_loader_params, shuffle=False,in_order=True)
validation_dataloader = DataLoader(Validation_Dataset, **data_loader_params, shuffle=False,in_order=True)
test_dataloader       = DataLoader(Test_Dataset, **data_loader_params, shuffle=False,in_order=True)

In [5]:
writeCSV(mask_model,test_dataloader,device,dataset,datasetType,data_aug_type)